In [1]:
import logging
from copy import deepcopy

from langchain_community.document_loaders import TextLoader
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_classic.storage import InMemoryStore
from langchain_classic.retrievers.parent_document_retriever import ParentDocumentRetriever
from langchain_classic.retrievers.multi_query import MultiQueryRetriever
from langchain_chroma import Chroma
from langchain_openai import ChatOpenAI
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_classic.chains import RetrievalQA

In [2]:
embeddings = HuggingFaceEmbeddings(model='BAAI/bge-m3', model_kwargs={'device':'cuda'}, encode_kwargs={'batch_size':8})
model = ChatOpenAI(model='gpt-5-mini', temperature=0.2)

In [3]:
loaders = [TextLoader('./data/How_to_invest_money.txt', encoding='utf-8')]
docs = []
for loader in loaders:
    docs.extend(loader.load())

In [4]:
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
child_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=50)

In [5]:
vectorstore = Chroma(collection_name='split_parents', embedding_function=embeddings)
docstore = InMemoryStore()

In [6]:
pc_retriever = ParentDocumentRetriever(
    docstore=docstore,
    vectorstore=vectorstore,
    parent_splitter=parent_splitter,
    child_splitter=child_splitter
)

pc_retriever.add_documents(docs)

In [7]:
print(len(list(docstore.yield_keys())))
print(vectorstore._collection.count())

219
1148


In [8]:
query = 'What are the types of investments?'
retrieved_docs = pc_retriever.invoke(query)
print(retrieved_docs[0].page_content)
print('--'*30)
sub_docs = vectorstore.similarity_search(query)
print(sub_docs[0].page_content)

There are five chief points to be considered in the selection of all
forms of investment. These are: (1) safety of principal and interest;
(2) rate of income; (3) convertibility into cash; (4) prospect of
appreciation in intrinsic value; (5) stability of market price.

Keeping these five general factors in mind, the present chapter will
discuss real-estate mortgages as a form of investment, both as adapted
to the requirements of private funds and of a business surplus.
------------------------------------------------------------
There are five chief points to be considered in the selection of all
forms of investment. These are: (1) safety of principal and interest;


In [9]:
logging.basicConfig()
logging.getLogger('langchain_classic.retrievers.multi_query').setLevel(logging.INFO)

In [10]:
mq_retriever = MultiQueryRetriever.from_llm(
    retriever=pc_retriever,
    llm=model
)

In [11]:
question = '주식 투자를 처음 시작하려면 어떻게 해야 하나요?'
unique_docs = mq_retriever.invoke(question)
print(f'{len(unique_docs)}개의 문서가 검색되었습니다.')

INFO:langchain_classic.retrievers.multi_query:Generated queries: ['주식 투자를 처음 시작하려면 어떤 준비와 절차가 필요하나요?', '초보자가 주식 투자를 시작할 때 알아야 할 기본 지식, 계좌 개설 방법, 추천 학습 자료는 무엇인가요?', '소액으로 주식 투자를 처음 시작할 때의 실전 투자 방법과 리스크 관리 요령을 알려주세요.']


7개의 문서가 검색되었습니다.


In [12]:
qa_chain = RetrievalQA.from_chain_type(
    llm=model,
    retriever=mq_retriever,
    chain_type='stuff',
    return_source_documents=True
)

In [13]:
result = qa_chain.invoke({'query':question})
print(result['result'])

INFO:langchain_classic.retrievers.multi_query:Generated queries: ['주식 투자를 처음 시작하는 초보자가 준비해야 할 단계와 필수 지식은 무엇인가요?', '주식 계좌 개설부터 첫 매수까지 실제로 따라할 수 있는 초보자용 가이드는?', '초보 투자자가 알아야 할 리스크 관리 방법, 종목 선정 기준 및 공부 자료 추천은?']


좋습니다 — 주식 투자를 처음 시작할 때 따라야 할 실용적이고 현실적인 단계들을 요약해 드리겠습니다. 아래 순서대로 하면 초보자의 실수 위험을 줄이고 장기적으로 안정적으로 자산을 키울 가능성을 높일 수 있습니다.

1) 투자 목적과 기간, 위험성향 정하기  
- 목적(예: 은퇴자금, 주택마련, 단기수익)과 투자기간을 분명히 하세요.  
- 감당할 수 있는 손실 수준(위험성향)을 미리 정해야 투자 스타일(공격형·중립형·안전형)이 나옵니다.

2) 기초 개념 이해하기 (주식과 채권의 차이 포함)  
- 주식: 회사의 지분. 배당·주가 상승은 기대할 수 있으나 원금 보장 없음.  
- 채권: 채무증서. 이자·원금 상환 약속(만기)이 있어 보통 안전성이 높음.  
- 투자 선택 시 고려할 5가지(책에서 제시된 핵심 기준): 원금 및 이자(배당)의 안전성, 수익률, 현금화(유동성), 내재가치의 상승 가능성, 시장가격의 안정성.

3) 기초 재무지식과 기업 분석법 배우기  
- 손익계산서·대차대조표·현금흐름표의 기본 항목을 익히세요.  
- P/E(주가수익비율), P/B, 배당수익률, 부채비율 같은 핵심 지표를 이해하세요.  
- 기업의 사업모델·경쟁우위·성장성·배당역사 등을 확인하세요.

4) 분산투자와 포트폴리오 구성  
- 개별주에 몰빵하지 마세요. 업종·지역·자산(주식·채권)으로 분산하세요.  
- 초보는 개별주보다 ETF(지수추종 상장지수펀드)나 인덱스펀드로 시작하는 것이 안전하고 비용효율적입니다.

5) 계좌 개설과 실전 준비  
- 신뢰할 수 있는 증권사(수수료·거래플랫폼·고객지원·세제혜택 등 고려)에서 계좌를 만드세요.  
- 비상금(생활비 3~6개월)과 고금리 부채 상환을 먼저 하고 남은 여유자금으로 투자하세요.  
- 처음엔 적은 금액으로 시작해 경험을 쌓으세요(달러코스트에버리징 추천).

6) 리스크 관리 원칙 지키기  
- 레버리지(빚내서 투자), 마진거래는 초반에 피하세요.  
- 포지션 크기 제한, 손절 규칙 등 본인만의 규율을 만드세요.  
-

In [14]:
def format_docs(docs):
    return '\n\n'.join(doc.page_content for doc in docs)

In [15]:
def create_virtual_doc_chain():
    system = '당신은 고도로 숙련된 AI입니다. 질문에 맞도록 적절하게 행동하세요.'
    user = '''
    주어진 질문 "{query}"에 대해 직접적으로 답변하는 가상의 문서를 생성하세요.
    문서의 크기는 {chunk_size} 언저리여야 합니다.
    해당 답변은 추후 다시 질문하는데 쓰일 예정임을 참고하세요.'''

    prompt = ChatPromptTemplate.from_messages(
        [('system', system),
         ('human', user)]
    )

    return prompt | model | StrOutputParser()

In [16]:
def create_retrieval_chain():
    return RunnableLambda(lambda x: mq_retriever.invoke(x['virtual_doc']))

In [17]:
def create_final_response_chain():
    template = '''다음 정보와 질문을 바탕으로 답변해주세요.
    컨텍스트: {context}
    질문: {question}
    답변:
    '''
    final_prompt = ChatPromptTemplate.from_template(template)

    return final_prompt | model | StrOutputParser()

In [18]:
def print_input_output(input_data, output_data, step_name):
    print(f'\n--- {step_name} ---')
    print(f'Input: {input_data}')
    print(f'Output: {output_data}')
    print('-' * 50)

In [19]:
def create_pipeline_with_logging():
    virtual_doc_chain = create_virtual_doc_chain()
    retrieval_chain = create_retrieval_chain()
    final_response_chain = create_final_response_chain()

    def virtual_doc_step(x):
        result = {'virtual_doc':virtual_doc_chain.invoke({'query':x['question'], 'chunk_size':200})}
        print_input_output(x, result, 'Virtual Doc Generation')
        return {**x, **result}
    
    def retrieval_step(x):
        result = {'retrieved_docs':retrieval_chain.invoke(x)}
        print_input_output(x, result, 'Document Retrieval')
        return {**x, **result}
    
    def context_formatting_step(x):
        result = {'context':format_docs(x['retrieved_docs'])}
        print_input_output(x, result, 'Context Formatting')
        return {**x, **result}
    
    def final_response_step(x):
        result = final_response_chain.invoke(x)
        print_input_output(x, result, 'Final Response Generation')
        return result
    
    pipeline = (
        RunnableLambda(virtual_doc_step)
        | RunnableLambda(retrieval_step)
        | RunnableLambda(context_formatting_step)
        | RunnableLambda(final_response_step)
    )

    return pipeline

In [20]:
pipeline = create_pipeline_with_logging()

In [21]:
question = '주식 시장의 변동성이 높을 때 투자 전략은 무엇인가요?'
response = pipeline.invoke({'question':question})
print(response)


--- Virtual Doc Generation ---
Input: {'question': '주식 시장의 변동성이 높을 때 투자 전략은 무엇인가요?'}
Output: {'virtual_doc': '주식 시장의 변동성이 높을 때는 포트폴리오 다각화, 현금 비중 확보, 정기적 리밸런싱, 분할매수(달러코스트에버리징), 방어적 섹터·고배당 종목 비중 확대, 손절 규칙 설정, 파생상품으로 헤지, 장기 관점 유지와 감정 통제가 핵심입니다. 리스크 허용범위 재점검과 수수료·세금 고려도 필수입니다.'}
--------------------------------------------------


INFO:langchain_classic.retrievers.multi_query:Generated queries: ['주식 시장 변동성이 높을 때 포트폴리오 관리에 적용할 실무 전략(다각화·현금 비중·정기적 리밸런싱·분할매수·방어섹터·고배당 비중 확대·손절 규칙·파생상품 헤지·장기 관점·감정 통제)과 단계별 체크리스트가 포함된 자료가 있나요?', '변동성 장에서 각 전략의 효과성과 근거를 비교한 연구·사례 자료(달러코스트에버리징, 리밸런싱 빈도, 고배당·방어섹터의 방어력, 파생상품 헤지의 비용·효과 등)를 추천해 주세요.', '리스크 허용범위 재점검 방법, 손절 기준 설정 규칙, 리밸런싱 빈도 결정법 및 수수료·세금이 포트폴리오 성과에 미치는 영향을 분석한 보고서나 실습용 계산 예제가 있는 문서를 찾고 싶습니다.']



--- Document Retrieval ---
Input: {'question': '주식 시장의 변동성이 높을 때 투자 전략은 무엇인가요?', 'virtual_doc': '주식 시장의 변동성이 높을 때는 포트폴리오 다각화, 현금 비중 확보, 정기적 리밸런싱, 분할매수(달러코스트에버리징), 방어적 섹터·고배당 종목 비중 확대, 손절 규칙 설정, 파생상품으로 헤지, 장기 관점 유지와 감정 통제가 핵심입니다. 리스크 허용범위 재점검과 수수료·세금 고려도 필수입니다.'}
Output: {'retrieved_docs': [Document(metadata={'source': './data/How_to_invest_money.txt'}, page_content='For the successful investment of money, however, a good deal more is\nrequired than the mere ability to select a safe security. That is only\none phase of the problem. Scientific investment demands a clear\nunderstanding of the fundamental distinctions between different classes\nof securities and strict adherence to the two cardinal principles,\ndistribution of risk and selection of securities in accordance with\nreal requirements.'), Document(metadata={'source': './data/How_to_invest_money.txt'}, page_content='It is plainly impossible within the limits of a small volume to consider\nthe needs of all classes of investors. 